In [ ]:
import os
import copy
from contextlib import contextmanager
import dataclasses

from DataclassInspector.inspector import Inspector

import numpy as np
import xarray as xr

from pymedphys.deliverydata import DeliveryData
from pymedphys.logfile import delivery_data_from_logfile

In [ ]:
# @contextmanager 
# def mutable(obj): 
#     cls = obj.__class__ 
#     obj.__class__ = object 
#     try: 
#         yield obj 
#     finally: 
#         obj.__class__ = cls 

In [ ]:
logfile_path = os.path.join(
    '..', '..',
    'tests', 'data', 'trf', 'integrity4',
    'INT4 18_12_12 08_28_23 Z [ICOM_ 7_03_2018 15_21_51] PF_G90..trf'
)
logfile_path

In [ ]:
delivery_data = delivery_data_from_logfile(logfile_path)

In [ ]:
MU = [0, 10, 20]

MLC = [
    [
        [3, -3],
        [3, -3],
        [3, -3]
    ],
    [
        [3, 3],
        [3, 3],
        [3, 3]
    ],
    [
        [-3, 3],
        [-3, 3],
        [-3, 3]
    ]
]

JAW = [
    [3, 3],
    [3, 3],
    [3, 3]
]

GANTRY = [0, 0, 0]
COLLIMATOR = [0, 0, 0]

In [ ]:
from pymedphys.xarray import xarray_dataclass

In [ ]:
# delivery_data

In [ ]:
test = xr.Dataset({
    'monitor_units': ('control_point', delivery_data.monitor_units),
    'gantry': ('control_point', delivery_data.gantry),
    'collimator': ('control_point', delivery_data.collimator),
    'mlc': (['control_point', 'leaf_pair', 'leaf_bank'], delivery_data.mlc),
    'jaw': (['control_point', 'jaw_bank'], delivery_data.jaw)
})

# test.__dict__

In [ ]:
type(exec('1'))

In [ ]:
from collections import namedtuple

DeliveryData1 = namedtuple(
    'DeliveryData',
    ['monitor_units', 'gantry', 'collimator', 'mlc', 'jaw'])


In [ ]:
class Foo(object):
    pass

foo = Foo()
foo.a = 3
Foo.b = property(lambda self: self.a + 1)
foo.b

In [ ]:
dataclasses.dataclass

In [ ]:
dataclasses.dataclass()

In [ ]:

# # https://github.com/python/cpython/blob/b9182aa7dad8991fc8768ae494b45b5f7c316aca/Lib/dataclasses.py#L364-L387
# def _create_attribute_fn(name, body, *, globals=None, locals=None):
#     if locals is None:
#         locals = {}
#     if globals is not None and '__builtins__' not in globals:
#         globals['__builtins__'] = builtins

#     body = '\n'.join(f'    {b}' for b in body)
#     txt = f'@property\ndef {name}(self):\n{body}'
    
#     print(txt)
    
#     exec(txt, globals, locals)
#     return locals[name]



def xarray_dataclass(cls):
    def __post_init__(self, *args, **kwargs):
        self._create_xarray()
        
    def _create_xarray(self):
        xarray = xr.Dataset({
            field.name: (field.type, getattr(self, '_' + field.name))
            for field in dataclasses.fields(self)
        })
        object.__setattr__(self, '_xarray', xarray)
        
    def __iter__(self):
        for field in dataclasses.fields(self):
            yield getattr(self, field.name)
            
    cls.__post_init__ = __post_init__
    cls._create_xarray = _create_xarray
    cls.__iter__ = __iter__
    
    cls = dataclasses.dataclass(frozen=True)(cls)
    
    for field in dataclasses.fields(cls):
        txt = '\n'.join([
            f'@property',
            f'def {field.name}(self):',
            f'    return self._xarray.{field.name}.data',
            '',
            f'@{field.name}.setter',
            f'def {field.name}(self, data):',
            f'    object.__setattr__(self, "_{field.name}", np.array(data))',
            f'    try:',
            f'        self._xarray.{field.name}.data = data',
            f'    except AttributeError:',
            f'        pass',
            '',
            f'cls.{field.name} = {field.name}'
        ])
        
        exec(txt)
    
    def to_tuple(self):
        return dataclasses.astuple(self)
    
#     def to_dict(self):
#         return dataclasses.asdict(self)
    
    def to_list(self):
        return list(self.to_tuple())
    
    def to_xarray(self):
        return self.deepcopy()._xarray
    
    def to_dataframe(self):
        return self.to_xarray().to_dataframe()

    def to_dict(self):
        return self.to_xarray().to_dict()

    def deepcopy(self):
        return copy.deepcopy(self)
            
    cls.to_tuple = to_tuple
    cls.to_dict = to_dict
    cls.to_xarray = to_xarray
    cls.to_dataframe = to_dataframe
    cls.deepcopy = deepcopy
    
    return cls


@xarray_dataclass
class Test:
    monitor_units: 'control_point'
    mlc: ['control_point', 'leaf_pair', 'leaf_bank']
    jaw: ['control_point', 'jaw_bank']
    gantry: 'control_point'
    collimator: 'control_point'


boo = Test(**delivery_data._asdict())
# a,b,c, d, e = boo
# a

# boo.xarray

boo.to_dataframe()

In [ ]:
@xarray_dataclass
class DummyDeliveryData:
    monitor_units: 'control_point'
    mlc: ['control_point', 'leaf_pair', 'leaf_bank']
    jaw: ['control_point', 'jaw_bank']
    gantry: 'control_point'
    collimator: 'control_point'

In [ ]:
new_container = DummyDeliveryData(MU, MLC, JAW, GANTRY, COLLIMATOR)

In [ ]:
new_container.to_tuple()

In [ ]:
# copy.deepcopy(boo)

In [ ]:
# inspected = Inspector(boo)
# display(inspected.code)

In [ ]:
{
    field.name: (field.type, )
    for field in dataclasses.fields(Test)
}

In [ ]:
dataclasses.fields(Test)

In [ ]:
boo.to_list()

In [ ]:
def xarray_dataclass(cls, fields):
    def wrapper(x):
        return func(x-1)
        
    
    return wrapper


@dec
def boo(x):
    return x

boo(3)

In [ ]:
@dataclasses.dataclass(frozen=True)
class DeliveryData2:
    monitor_units: np.ndarray
    gantry: np.ndarray
    collimator: np.ndarray
    mlc: np.ndarray
    jaw: np.ndarray
    
    def __iter__(self):
        for field in dataclasses.fields(self):
            yield getattr(self, field.name)


In [ ]:
DeliveryData3 = dataclasses.make_dataclass('DeliveryData3',
    ['monitor_units', 'gantry', 'collimator', 'mlc', 'jaw'], frozen=True)

DeliveryData3

In [ ]:
[
    field.name
    for field in dataclasses.fields(DeliveryData3)
]

In [ ]:
test = DeliveryData2(*delivery_data)

In [ ]:
# DeliveryData2(*test)

In [ ]:
a,b,c,d,e = test
# a

In [ ]:
# delivery_data